In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
#create test and tarin data and visualize them
df = pd.read_csv("video_games_sales.csv")
df = df.sort_values(by='Year_of_Release', ascending = True)
df = df.drop(df[(df.Global_Sales < 3) | (df.Year_of_Release < 2000) ].index)
# print(len(df))
train=df[0:132] 
test=df[132:] 
df.index = df.Year_of_Release 
train.index = train.Year_of_Release.astype(int)
test.index = test.Year_of_Release 
# plt.figure(figsize=(12,8))
# train.Global_Sales.plot(title= 'sales', fontsize=14)
# test.Global_Sales.plot(title= 'sales', fontsize=14)

In [ ]:
#Naive forecasting based on previous value
dd= np.asarray(train.Global_Sales)
y_hat = test.copy()
y_hat['naive'] = dd[len(dd)-1]
plt.figure(figsize=(16,8))
plt.plot(train.index, train['Global_Sales'], label='Train')
plt.plot(test.index,test['Global_Sales'], label='Test')
plt.plot(y_hat.index,y_hat['naive'], label='Naive Forecast')
plt.legend(loc='best')
plt.title("Naive Forecast")
plt.show()
rms = sqrt(mean_squared_error(test.Global_Sales, y_hat.naive))
print(rms)

In [ ]:
#simple average
y_hat_avg = test.copy()
y_hat_avg['avg_forecast'] = train['Global_Sales'].mean()
plt.figure(figsize=(16,8))
plt.plot(train['Global_Sales'], label='Train')
plt.plot(test['Global_Sales'], label='Test')
plt.plot(y_hat_avg['avg_forecast'], label='Average Forecast')
plt.legend(loc='best')
plt.show()
rms = sqrt(mean_squared_error(test.Global_Sales, y_hat_avg.avg_forecast))
print(rms)

In [ ]:
#moving average
y_hat_avg = test.copy()
y_hat_avg['moving_avg_forecast'] = train['Global_Sales'].rolling(50).mean().iloc[-1]
plt.figure(figsize=(16,5))
plt.plot(train['Global_Sales'], label='Train')
plt.plot(test['Global_Sales'], label='Test')
plt.plot(y_hat_avg['moving_avg_forecast'], label='Moving Average Forecast')
plt.legend(loc='best')
plt.show()
rms = sqrt(mean_squared_error(test.Global_Sales, y_hat_avg.moving_avg_forecast))
print(rms)

In [ ]:
#Simple Exponential Smoothing
y_hat_avg = test.copy()
fit2 = SimpleExpSmoothing(np.asarray(train['Global_Sales'])).fit(smoothing_level=0.5,optimized=False)
y_hat_avg['SES'] = fit2.forecast(len(test))
plt.figure(figsize=(16,8))
plt.plot(train['Global_Sales'], label='Train')
plt.plot(test['Global_Sales'], label='Test')
plt.plot(y_hat_avg['SES'], label='SES')
plt.legend(loc='best')
plt.show()
rms = sqrt(mean_squared_error(test.Global_Sales, y_hat_avg.SES))
print(rms)

In [ ]:
#Holt's Linear trend method 
# train.index = pd.to_datetime(train.Year_of_Release)
sm.tsa.seasonal_decompose(train.Global_Sales, freq=30).plot()
result = sm.tsa.stattools.adfuller(train.Global_Sales)
plt.show()

In [ ]:
# Holt's Linear trend method 
y_hat_avg = test.copy()

fit1 = Holt(np.asarray(train['Global_Sales'])).fit(smoothing_level = 0.3,smoothing_slope = 0.1)
y_hat_avg['Holt_linear'] = fit1.forecast(len(test))

plt.figure(figsize=(16,8))
plt.plot(train['Global_Sales'], label='Train')
plt.plot(test['Global_Sales'], label='Test')
plt.plot(y_hat_avg['Holt_linear'], label='Holt_linear')
plt.legend(loc='best')
plt.show()

In [ ]:
#holt's seasonal winter method
y_hat_avg = test.copy()
fit1 = ExponentialSmoothing(np.asarray(train['Global_Sales']) ,seasonal_periods=5 ,trend='mul', seasonal='mul',).fit()
y_hat_avg['Holt_Winter'] = fit1.forecast(len(test))
plt.figure(figsize=(16,8))
plt.plot( train['Global_Sales'], label='Train')
plt.plot(test['Global_Sales'], label='Test')
plt.plot(y_hat_avg['Holt_Winter'], label='Holt_Winter')
plt.legend(loc='best')
plt.show()

In [3]:
#Seasonal ARIMA 
y_hat_avg = test.copy()
# train.Year_of_Release = train.Year_of_Release.astype(int)
# test.index = pd.to_datetime(test.Year_of_Release)
# train.Year_of_Release
fit1 = sm.tsa.statespace.SARIMAX(train.Global_Sales, order=(2, 1, 4),seasonal_order=(0,1,1,7)).fit()
y_hat_avg['SARIMA'] = fit1.predict(start="2000", end="2007", dynamic=True, axis=1)
plt.figure(figsize=(16,8))
plt.plot( train['Global_Sales'], label='Train')
plt.plot(test['Global_Sales'], label='Test')
plt.plot(y_hat_avg['SARIMA'], label='SARIMA')
plt.legend(loc='best')
plt.show()

C:\Users\lenovo\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:215: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


ValueError: cannot reindex from a duplicate axis